In [1]:
import os
from diffusers import AutoPipelineForText2Image
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid
import torch
from PIL  import Image 

import torchvision.transforms as transforms
import numpy as np 

import sys
from io import StringIO
import os
import utils_img

from PIL import Image
from torchvision.utils import save_image



def  img2img(imgs):
    c, h,w = imgs.shape[1], imgs.shape[-2], imgs.shape[-1]


    # 以下代码引入SDXL模型
    model= "/ssd-sata1/hqq/stable_signature/stable-diffusion-2-1-base"
    pipeline_img2img = AutoPipelineForImage2Image.from_pretrained(
            model, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
    ).to("cuda")
    #pipeline_text2image.model.config.image_size = imgs.shape[-1]
    prompt = "red sky"

    for i in range(imgs.shape[0]):
        img=imgs[i]
        #img=Image.fromarray(img.numpy())
        img = img

        img2 = pipeline_img2img(prompt, image=img, 
                                strength=0.8, guidance_scale=10.5,height=imgs.shape[-2],width=imgs.shape[-1]).images[0]
        img2.save('t.png')

/home/huqiqi/anaconda3/envs/WM_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/huqiqi/anaconda3/envs/WM_env/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/huqiqi/anaconda3/envs/WM_env/lib/python3.9/site-packages/diffusers/models/vq_model.py:20: FutureWarning: `VQEncoderOutput` is deprecated and will be removed in version 0.31. Importing `VQEncoderOutput` from `diffusers.models.vq_model` is deprecated and this will be removed in a future version. Please use `from diffusers.models.autoencoders.vq_model import VQEncoderOutput`, instead.
  deprecate("VQEncoderOutput", "0.31", deprecation_message)
/home/hu

In [7]:
from diffusers.utils import load_image, make_image_grid

image1=load_image("datasets/train_dir/000000022396.jpg")
image2= load_image("datasets/train_dir/000000026564.jpg")
# # 使用Pillow裁剪图像
# left, upper, right, lower = 0, 0, 256, 256  # 裁剪区域的坐标
# image1 = image1.crop((left, upper, right, lower))

train_transform = transforms.Compose([
        transforms.RandomResizedCrop(512),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomCrop(256, padding=None, pad_if_needed=False, fill=0, padding_mode='constant'),
        #transforms.CenterCrop((512,512)),
        transforms.ToTensor(),
        #utils_img.normalize_rgb,
    ])

tensor1= train_transform(image1)
tensor2=train_transform(image2)

tensor1=tensor1.unsqueeze(0)
tensor2=tensor2.unsqueeze(0)

tensor= torch.cat((tensor1, tensor2), dim=0)

print(tensor.shape)



torch.Size([2, 3, 512, 512])


In [8]:
c, h,w = tensor.shape[1], tensor.shape[-2], tensor.shape[-1]


    # 以下代码引入SDXL模型
model= "/ssd-sata1/hqq/stable_signature/stable-diffusion-2-1-base"
pipeline_img2img = AutoPipelineForImage2Image.from_pretrained(
            model, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
    ).to("cuda")

Loading pipeline components...: 100%|██████████| 6/6 [00:01<00:00,  5.45it/s]


In [12]:
prompt=' '
img2 = pipeline_img2img(prompt, image=tensor, 
                                strength=0.8, guidance_scale=10.5,height=512,width=512,).images[0]

  0%|          | 0/40 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (8192) must match the size of tensor b (4096) at non-singleton dimension 1